In [ ]:
# Import necessary libraries for data curation, visualization, and dimensionality reduction.
# If you are re-running this notebook, it could be a good practice to restart kernel and clear outputs of all cells.
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import umap 
import os


In [ ]:
# Define a custom color palette for visualizing different populations in the dataset.
custom_palette = {
    "EAD_A": "#cada45",  # Green
    "EAD_B": "#d4a2e1",  # Purple
    "EEP": "#55e0c6",    # Blue
    "USA": "#f0b13c",    # Orange
}

In [ ]:
# List the distance files to process
dist_files = [
    'input_files/A.dist',
    'input_files/B.dist',  
    'input_files/C.dist'
]

# Load population data
population_names = pd.read_csv('input_files/pop_info.txt', sep='\t', header=0)

# Prepare a dataframe to store the analysis results.
Data_Struct = population_names

In [ ]:
# Define parameter ranges for t-SNE.
# We set the maximum to be half of the number of individuals in the dataset.
# We set the minimum to be the smallest number of individuals in given populations.
# We then took a middle value between the min and max.
perplexity_values = (5, 10, 23)

# Define parameter ranges for UMAP.
# Default value is 0.1.
mindists = (0.01, 0.1, 0.5)

# We set the maximum to be half of the number of individuals in the dataset.
# We set the minimum to be the smallest number of individuals in given populations.
# We then took a middle value between the min and max.
n_neighbors_nums = (5, 10, 23)

In [ ]:
# Iterate over each distance matrix file
for filename in dist_files:
    print(f'Processing {filename}...')
    distance_matrix = pd.read_csv(filename, sep='\t', header=None, index_col=0)
    dist_mat_np = distance_matrix.to_numpy()

    # Prepare a dataframe to store the analysis results.
    Data_Struct = population_names.copy()

    # Perform t-SNE
    for perp in perplexity_values:
        np.random.seed(88)
        proj_tsne = TSNE(n_components=2, perplexity=perp).fit_transform(distance_matrix)
        Data_Struct[f'tSNE-1 perp{perp}'] = proj_tsne[:, 0]
        Data_Struct[f'tSNE-2 perp{perp}'] = proj_tsne[:, 1]
    
   # Perform UMAP.
    for nn in n_neighbors_nums:
        for mind in mindists:
            proj_umap = umap.UMAP(n_components=2, n_neighbors=nn, min_dist=mind,random_state=88).fit_transform(distance_matrix)
            Data_Struct[f'UMAP-1 numn{nn} mindist{mind}'] = proj_umap[:, 0]
            Data_Struct[f'UMAP-2 numn{nn} mindist{mind}'] = proj_umap[:, 1]
    
    # Prepare and plot the final visualizations for both t-SNE and UMAP.
    n_rows = len(perplexity_values)
    n_cols = 1 + len(n_neighbors_nums)  # Adding one column for t-SNE and the rest for UMAP.
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))
    fig.suptitle(f"input: {filename}", fontsize=14)

    # Generate plots for t-SNE and UMAP, adjusting labels and legends appropriately.
    # t-SNE plots (first column)
    for i, perp in enumerate(perplexity_values):
        sns.scatterplot(ax=axs[i, 0], data=Data_Struct, x='tSNE-1 perp' + str(perp), y='tSNE-2 perp' + str(perp), s=300, hue='Population', palette=custom_palette, legend=False)
        axs[i, 0].set_xlabel('tSNE-1')
        axs[i, 0].set_ylabel('tSNE-2')
        axs[i, 0].set_xticks([])
        axs[i, 0].set_yticks([])
        # Set the title differently for the first plot
        if i == 0:
            axs[i, 0].set_title('tSNE / Perplexity = ' + str(perp))
        else:
            axs[i, 0].set_title('Perplexity = ' + str(perp))


    # UMAP plots (next columns)
    for j, nn in enumerate(n_neighbors_nums):
        for i, mind in enumerate(mindists):
            is_last_plot = (i == len(mindists) - 1) and (j == len(n_neighbors_nums) - 1)
            sns.scatterplot(ax=axs[i, j + 1], data=Data_Struct, x='UMAP-1 numn' + str(nn) + ' mindist' + str(mind), y='UMAP-2 numn' + str(nn) + ' mindist' + str(mind), s=300, hue='Population', palette=custom_palette, legend=is_last_plot)
        
            if i == 0:
                axs[i, j + 1].set_title('UMAP / n_neighbours = ' + str(nn))
                axs[i, j + 1].set_xlabel('UMAP-1')

            # Set the y-axis label differently for the first column of UMAP plots
            if j == 0:
                axs[i, j + 1].set_ylabel('min_dist = ' + str(mind))
            else:
                axs[i, j + 1].set_ylabel('UMAP-2')

            axs[i, j + 1].set_xticks([])
            axs[i, j + 1].set_yticks([])
        
            # Adjust the legend for the last UMAP plot (bottom-right)
            if is_last_plot:
                axs[i, j + 1].legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Population')
    
    output_filename = f"{os.path.splitext(os.path.basename(filename))[0]}.png"
    plt.savefig(output_filename, format='png', dpi=300, transparent=False, facecolor='white')
    print(f'Saved plot: {output_filename}\n')

    #You can output also as pdf:
    #output_filename = f"{os.path.splitext(os.path.basename(filename))[0]}.pdf"
    #plt.savefig(output_filename, format='pdf', dpi=300, transparent=False, facecolor='white')

    
